In [ ]:
import toml
import pygsheets
import requests
import Bio.Restriction as Restriction
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.design as design
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.viennarna as viennarna
import paulssonlab.cloning.thermodynamics as thermodynamics
import paulssonlab.cloning.primers as primers
import paulssonlab.api.geneious as geneious

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
geneious_sessionmaker = geneious.connect(**config["geneious"])

In [ ]:
reg = registry.Registry(
    gc,
    config["registry"]["folder"],
    geneious_sessionmaker=geneious_sessionmaker,
    geneious_folder="registry",
)

# Config

In [ ]:
olib_oligos = reg[("oLIB", "oligos")]
plib_plasmids = reg[("pLIB", "plasmids")]
plib_maps = reg[("pLIB", "maps")]
lib_strains = reg[("LIB", "strains")]
flib_fragments = reg[("fLIB", "fragments")]
part_types = reg[("fLIB", "fragments", "Part types")]

# Deg tags

## Config

In [ ]:
# degtags
degtag_part_type = "ClpXP_tag"
degtag_overhangs = workflow.overhangs_for(part_types[degtag_part_type])
library_reverse_primer = "oLIB46"
degtag_enzyme = Restriction.BbsI
placeholder_enzyme = Restriction.BbsI
num_random_bases = 6  # random bases to add between enzyme binding site and end of DNA
tail_length_aa = 3  # aa
tail_length = tail_length_aa * 3  # nt
sspb_length_aa = 6  # aa (part of the SspB binding site that is mutagenized with NNK's; note we keep first two alanines fixed)
sspb_length = sspb_length_aa * 3  # nt
head_length = 6  # nt
reference = "Andersen, J. B., Sternberg, C., Poulsen, L. K., Bjørn, S. P., Givskov, M., & Molin, S. (1998). New unstable variants of green fluorescent protein for studies of transient gene expression in bacteria. Applied and environmental microbiology, 64(6), 2240-2246."

# placeholders
tm_binding = 60
tm_homology = 55
min_mfe = None

## Data

In [ ]:
# FROM: Andersen, J. B., Sternberg, C., Poulsen, L. K., Bjørn, S. P., Givskov, M., & Molin, S. (1998). New unstable variants of green fluorescent protein for studies of transient gene expression in bacteria. Applied and environmental microbiology, 64(6), 2240-2246.
# paper gives reverse-complement sequences
degtags_wt_rc = {
    "LAA": "AGCTGCTAAAGCGTAGTTTTCGTCGTTTGCTGC",
    "AAV": "AACTGCTGCAGCGTAGTTTTCGTCGTTTGCTGC",
    # "LVA": "AGCTACTAAAGCGTAGTTTTCGTCGTTTGCTGC", # paper implies it behaves similarly to LAA
    "ASV": "AACTGATGCAGCGTAGTTTTCGTCGTTTGCTGC",
}
degtags_wt_rc = {name: Seq(seq) for name, seq in degtags_wt_rc.items()}
degtags_wt = {name: sequence.reverse_complement(seq) for name, seq in degtags_wt_rc.items()}

In [ ]:
degtags_wt["AAV"].translate()

## Degtags

In [ ]:
degtag_normalized = {name: workflow.normalize_seq(seq) for name, seq in degtags_wt.items()}
degtag_prefixes = {
    name: seq[: seq.find(degtag_overhangs[0])] for name, seq in degtag_normalized.items()
}
degtag_prefixes_list = list(degtag_prefixes.values())
degtag_prefix = degtag_prefixes_list[0]
assert all(s == degtag_prefix for s in degtag_prefixes_list)

## Placeholder

In [ ]:
# part_names = ["sigW", "rsiW", "ECF20_992", "AS20_992", "sfGFP"]  # TODO: pick correct FPs
#part_names = ["AM19_LacIAM"]#, "AM19_TetR"]
#part_names = ["AM19_TetR"]
#part_names = ["mScarlet-I"]
part_names = ["sigW"]

In [ ]:
degtag_placeholder = degtag_prefix + design.golden_gate_placeholder(
#    placeholder_enzyme, None, *degtag_overhangs
    Restriction.BsaI, None, *degtag_overhangs
)
# remove downstream overhang because it is already in the storage vector
degtag_placeholder = degtag_placeholder[: -len(degtag_overhangs[1])]

In [ ]:
degtag_placeholder

## Design

In [ ]:
%%time

row_base = {"Author": "Jacob Quinn Shenker", "Date": workflow.date()}

oligo_base = {
    **row_base,
    "Order date": workflow.date(),
    "Vendor": "Genewiz",
    "Type": "Primer",
}

plasmid_base = {
    #"Origin": "pUC",
    #"Marker": "amp",
    #"Reference": reference,
}

strain_base = {
    "Species": "E. coli",
    "Background": "MG1655",
    "Parent": "MG1655",
    #"Marker": "amp",
}

fragment_base = {
    **row_base,
    "Species/codon usage": "E. coli",
    #"Reference": reference,
}

part_enzyme = Restriction.BsaI
part_type = "CDS_CD"
base_rows = dict(oligo=oligo_base, plasmid=plasmid_base, strain=strain_base, fragment=fragment_base)
# plasmid_description = f"Golden gate part for Adam Meyer's {name} CDS followed by a bicistronic CDS placeholder."
# part_description = f"Adam Meyer's {name} CDS followed by a bicistronic CDS placeholder. Sequence taken unmodified from color-coded region in the supplemental information for Meyer 2019. Sequences PCRed from {id_} ({ajm})."
descriptions = dict(oligo="", plasmid="", fragment="")
#flank = "ggggggggg"
flank = degtag_placeholder

for part_name in part_names:
    workflow.insert_cds_part_flank(
        f"{part_name}+ClpXP_tag",
        part_name,
        flank,
        part_type,
        part_enzyme,
        descriptions,
        base_rows,
        reg,
        olib_oligos,
        plib_plasmids,
        plib_maps,
        lib_strains,
        flib_fragments,
        part_types,
        stop_location="downstream",
        location="downstream",
        tm_binding=tm_binding,
        tm_homology=tm_homology,
        min_mfe=min_mfe,
    )

In [ ]:
olib_oligos.local

In [ ]:
olib_oligos.rollback()
plib_plasmids.rollback()
plib_maps.rollback()
lib_strains.rollback()
flib_fragments.rollback()

# Bicistronic

## Config

In [ ]:
bicistronic_enzyme = Restriction.BsmBI
bicistronic_overhangs = workflow.overhangs_for(part_types["3G_BD"])

## Design

In [ ]:
bicistronic_placeholder = design.golden_gate_placeholder(
    bicistronic_enzyme, None, *bicistronic_overhangs
)

In [ ]:
bicistronic_placeholder